**all neccessary library imports**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from time import time
import pickle

**import dataset and clean up data**

In [2]:
query_data = pd.read_csv("./data/sqli.csv", encoding='utf-8')

# remove duplicates from dataset
query_data.drop_duplicates(inplace=True)

# handle any missing value on dataset
query_data.dropna(inplace=True)

# check if there any null value in the dataset
query_data.isnull().sum()

Query    0
Label    0
dtype: int64

In [3]:
print(f"Safe Queries: {query_data['Label'].value_counts()[0]}")
print(f"Malicious Queries: {query_data['Label'].value_counts()[1]}")
print(f"Data shape: {query_data.shape}")
query_data.head(10)

Safe Queries: 19529
Malicious Queries: 11378
Data shape: (30907, 2)


,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
5,select name from syscolumns where id = ...,1
6,select * from users where id = 1 +$+ or 1 =...,1
7,"1; ( load_file ( char ( 47,101,116,99,47...",1
8,select * from users where id = '1' or ||/1 ...,1
9,select * from users where id = '1' or \.<\ ...,1


#### data preprocessing

In [4]:
X = query_data['Query']
y = query_data['Label']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1) 
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [6]:
vectorizer.get_feature_names_out()

array(['00', '000001', '000003', ..., 'zzip', 'zzkx', 'ã½'], dtype=object)

In [7]:
X_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
X_test.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

> test different model

In [9]:
# training a model using decision tree algorithm
d_model = DecisionTreeClassifier()
start_time = time()
d_model.fit(X_train, y_train)
end_time = time()
d_pred = d_model.predict(X_test)
d_model_accuracy = accuracy_score(y_test, d_pred)
d_model_f1_score = f1_score(y_test, d_pred)
print(f"Training Time: {end_time-start_time:.2f} seconds")

Training Time: 5.72 seconds


In [10]:
# training a model using random forest tree algorithm
r_model = RandomForestClassifier()
start_time = time()
r_model.fit(X_train, y_train)
end_time = time()
r_pred = r_model.predict(X_test)
r_model_accuracy = accuracy_score(y_test, r_pred)
r_model_f1_score = f1_score(y_test, r_pred)
print(f"Training Time: {end_time-start_time:.2f} seconds")

Training Time: 13.89 seconds


In [11]:
# training a model using Support Vector Machine (SVM) algorithm
s_model = SVC()
start_time = time()
s_model.fit(X_train, y_train)
end_time = time()
s_pred = s_model.predict(X_test)
s_model_accuracy = accuracy_score(y_test, s_pred)
s_model_f1_score = f1_score(y_test, s_pred)
print(f"Training Time: {end_time-start_time:.2f} seconds")

Training Time: 70.78 seconds


> chosen naive bayes model

In [12]:
# training a naive Bayes model
n_model = MultinomialNB()
start_time = time()
n_model.fit(X_train,y_train)
end_time = time()
n_pred = n_model.predict(X_test)
n_model_accuracy = accuracy_score(y_test, n_pred)
n_model_f1_score = f1_score(y_test, n_pred)
print(f"Training Time: {end_time-start_time:.2f} seconds")

Training Time: 0.01 seconds


#### model evaluation

#### Formulas

> accuracy = (TP + TN) / (TP + TN + FP + FN)

> Precision = TP / (TP + FP)

> Recall = TP / (TP + FN)

> F1-score = 2 * (Precision * Recall) / (Precision + Recall)

In [13]:
from sklearn.metrics import confusion_matrix

def model_eval(model_pred, model_name:str, y_test=y_test):
    confusion = confusion_matrix(y_test, model_pred)

    TP = confusion[1,1] # true positive
    TN = confusion[0,0] # true negative
    FP = confusion[1,0] # false positive
    FN = confusion[0,1] # false negative
    
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP+FN)
    f1_score = 2 * (precision * recall) / (precision+recall)
    
    print(f"Model evaluation of {model_name}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")
    print("--------------------------------------")


In [14]:
model_eval(d_pred, "decision tree")
model_eval(r_pred, "random forest")
model_eval(s_pred, "support vection machine")
model_eval(n_pred, "naive bayes")

Model evaluation of decision tree
Accuracy: 0.7790359107085085
Precision: 0.9219107551487414
Recall: 0.6447289457891578
F1-score: 0.7587992937021778
--------------------------------------
Model evaluation of random forest
Accuracy: 0.8011431036342068
Precision: 0.9939931350114416
Recall: 0.6559078897697245
F1-score: 0.7903115760745963
--------------------------------------
Model evaluation of support vection machine
Accuracy: 0.8208778173190985
Precision: 0.9711098398169337
Recall: 0.6851664984863775
F1-score: 0.8034552124008992
--------------------------------------
Model evaluation of naive bayes
Accuracy: 0.9682950501455839
Precision: 0.9407894736842105
Recall: 0.9742298578199052
F1-score: 0.9572176949941793
--------------------------------------


In [15]:
# save this training model for futher use
pickle.dump(n_model, open("model_n.pkl", "wb"))

In [16]:
# load model and test 
model = pickle.load(open("model_n.pkl", "rb"))

with open("sql_query.txt", "r") as file:
    lines = file.readlines()
    
user_given_query_data = pd.DataFrame(lines, columns=['Query'])
user_given_query_data = user_given_query_data.replace("\n", "", regex=True)

sql_query = vectorizer.transform(user_given_query_data['Query'])
results = model.predict(sql_query)

malicious = 0
safe = 0
for result in results:
    if result == 1:
        malicious = malicious + 1
    else:
        safe = safe + 1
        
print(f"Flag out malicious queries as malicious: {malicious}")
print(f"Flag out malicious queries as safe: {safe}")

Flag out malicious queries as malicious: 5
Flag out malicious queries as safe: 0


In [17]:
user_given_query_data.head(5)

,Query
0,"'UNION SELECT user, password FROM users#"
1,"1 or 1=1 UNION SELECT user, password FROM users#"
2,"1'UNION SELECT user, password FROM users#"
3,1' or sleep(4)#
4,"1' benchmark(10000000,MD5(1))#"


#### manual testing
Perform a manual testing on a website that has input validation with these same 5 malicious queries.

> Result of manual testing

```
| low  | Medium | High |

|  1   |    1   |   0  |

|  0   |   1    |   0  |

|  1   |   0    |   1  |

|  1   |   0    |   1  |

|  1   |   0    |   1  |

```